<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/reports/items_and_publication_date_aggregations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CHPL - Collection Analysis - **Items -- Publication Dates & Circulation**

<img src="https://ilsweb.cincinnatilibrary.org/img/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="300"/>

Show item counts from the past 5 decades by item type and faceted by decade, and then by the last 5 years

Data sources

* https://collection-analysis.cincy.pl/

In [1]:
#@title
!pip install rich

In [69]:
#@title
import pandas as pd
import numpy as np
import altair as alt

# if we want to save output as .png use this
# NOTE: use !pip install -U altair_saver above to download / install first
# from altair_saver import save
from urllib.parse import urlencode

import requests
from requests.adapters import HTTPAdapter, Retry

from time import sleep

# from rich import print
from rich.jupyter import print

base_url = 'https://collection-analysis.cincy.pl/current_collection'

# this will allow us to try multiple times to make a request
s = requests.Session()
retries = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[502, 503, 504]
)

s.mount('https://', HTTPAdapter(max_retries=retries))

In [ ]:
#@title
sql = """\
-- group titles (bib records) by the decade in which they were published
select
  cast(round(publish_year / 10, 1) * 10 as integer) AS decade,
  case
    when item.item_format = 'New Release DVDs' then 'DVD/Videocassette'
    else item.item_format
  end as item_format,
  count(*) AS count_items
from
  bib
  join item on item.bib_record_num = bib.bib_record_num
where
  bib.publish_year >= 1980
  and bib.publish_year <= 2022
  and item_format in (
    'Book',
    'Juvenile Book',
    'Reference Book',
    'DVD/Videocassette',
    'Music on CD',
    'Teen Book',
    'Magazine',
    'Large Print Book',
    'Microfilm',
    'Rare Book',
    'Book on CD',
    'Microfiche',
    'Music Score',
    'Reference Juvenile Book',
    'LP Record',
    'New Release DVDs',
    'Juvenile Magazine',
    'Reference Magazine',
    'Juvenile Book on CD',
    'Leased Book'
  )
group by
  1,
  2
order by
  1,2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

In [ ]:
#@title
print('Click the link below to view the raw data:', base_url + '?' + urlencode({'sql':sql}), sep='\n')

Click the link below to view the raw data:
https://collection-analysis.cincy.pl/current_collection?sql=--+group+titles+%28bib+records%29+by+the+decade+in+which+they+were+published%0Aselect%0A++cast%28round%28publish_year+%2F+10%2C+1%29+%2A+10+as+integer%29+AS+decade%2C%0A++case%0A++++when+item.item_format+%3D+%27New+Release+DVDs%27+then+%27DVD%2FVideocassette%27%0A++++else+item.item_format%0A++end+as+item_format%2C%0A++count%28%2A%29+AS+count_items%0Afrom%0A++bib%0A++join+item+on+item.bib_record_num+%3D+bib.bib_record_num%0Awhere%0A++bib.publish_year+%3E%3D+1980%0A++and+bib.publish_year+%3C%3D+2022%0A++and+item_format+in+%28%0A++++%27Book%27%2C%0A++++%27Juvenile+Book%27%2C%0A++++%27Reference+Book%27%2C%0A++++%27DVD%2FVideocassette%27%2C%0A++++%27Music+on+CD%27%2C%0A++++%27Teen+Book%27%2C%0A++++%27Magazine%27%2C%0A++++%27Large+Print+Book%27%2C%0A++++%27Microfilm%27%2C%0A++++%27Rare+Book%27%2C%0A++++%27Book+on+CD%27%2C%0A++++%27Microfiche%27%2C%0A++++%27Music+Score%27%2C%0A++++%27Referen

In [ ]:
#@title
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'item_format',
        axis=alt.Axis(title='item format', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_items:Q',
        title = 'count total items',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'item_format',
        # legend=None
    ),
    tooltip=['decade', 'item_format', 'count_items']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=270,
).facet(
    facet='decade',
    columns=5
)

chart.display()

alt.FacetChart(...)

In [ ]:
#@title
sql = """\
select
  publish_year,
  case
    when item.item_format = 'New Release DVDs' then 'DVD/Videocassette'
    else item.item_format
  end as item_format,
  count(*) AS count_items
from
  bib
  join item on item.bib_record_num = bib.bib_record_num
where
  bib.publish_year >= 2018
  and bib.publish_year <= 2022
  and item_format in (
    'Book',
    'Juvenile Book',
    'Reference Book',
    'DVD/Videocassette',
    'Music on CD',
    'Teen Book',
    'Magazine',
    'Large Print Book',
    'Microfilm',
    'Rare Book',
    'Book on CD',
    'Microfiche',
    'Music Score',
    'Reference Juvenile Book',
    'LP Record',
    'New Release DVDs',
    'Juvenile Magazine',
    'Reference Magazine',
    'Juvenile Book on CD',
    'Leased Book'
  )
group by
  1,
  2
order by
  1,2
"""

df = pd.read_json(
    requests.get(
      url=base_url + '.json',
      params={
        'sql': sql,
        '_shape': 'array'
      }
    ).content
)

# df.head()

In [ ]:
#@title
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'item_format',
        axis=alt.Axis(title='item format', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_items:Q',
        title = 'count total items',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'item_format',
        # legend=None
    ),
    tooltip=['publish_year', 'item_format', 'count_items']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=270,
).facet(
    facet='publish_year',
    columns=5
)

chart.display()

alt.FacetChart(...)

# Find Items by Create Date that have 0 Circulations

In [61]:
#@title
%%time



url = """\
https://collection-analysis.cincy.pl/current_collection\
/item_view.json?\
&checkout_total__exact=0\
&creation_date__gte={}\
&creation_date__lte={}\
&item_format__notin=Downloadable+Audiobook%2C+Downloadable+Book%2C+Downloadable+Video%2C+Downloadable+Music%2C+Web+Document%2C+Rare+Book\
""".format('2021-03-01', '2021-09-01')
print(url)

results = s.get(url=url)
df = pd.DataFrame()
df = df.append(results.json()['rows'], ignore_index=True)

print(results.json()['human_description_en'])
print('filtered_table_rows_count:', results.json()['filtered_table_rows_count'])

url = results.json()['next_url']
track_range = int( int(results.json()['filtered_table_rows_count']) / int(results.json()['next']))
# print(track_range)
print('---')
for n in track(range(track_range), description="Processing the above query..."):
  results = s.get(url=url)
  df = df.append(results.json()['rows'])
  url = results.json()['next_url']

# replace the column names ...
df.columns = results.json()['columns']

https://collection-analysis.cincy.pl/current_collection/item_view.json?&checkout_total__exact
=0&creation_date__gte=2021-03-01&creation_date__lte=2021-09-01&item_format__notin=Downloadabl
e+Audiobook%2C+Downloadable+Book%2C+Downloadable+Video%2C+Downloadable+Music%2C+Web+Document%
2C+Rare+Book

where checkout_total = 0, creation_date ≥ 2021-03-01, creation_date ≤ 2021-09-01 and 
item_format not in ["Downloadable Audiobook", "Downloadable Book", "Downloadable Video", 
"Downloadable Music", "Web Document", "Rare Book"]

filtered_table_rows_count: 23304

---

Output()

CPU times: user 4.2 s, sys: 240 ms, total: 4.44 s
Wall time: 45.3 s


In [59]:
#@title
print(df.shape)
print(df.head())

(23304, 23)

item_record_num  volume_record_num  bib_record_num creation_date  \
0         11077892          1517125.0         1211762    2021-03-01   
1         11077893          1517125.0         1211762    2021-03-01   
2         11077894          1517125.0         1211762    2021-03-01   
3         11077895          1513991.0         1391284    2021-03-01   
4         11077896          1513991.0         1391284    2021-03-01   

         barcode item_format location_code  \
0  A000071152179    Magazine         mmamg   
1  A000071152203    Magazine         2mamh   
2  A000071152195    Magazine         chamg   
3  A000071152229    Magazine         nramg   
4  A000071152211    Magazine         2mamh   

                                       location_name    branch_name  \
0                          Mariemont Adult Magazines      Mariemont   
1  Main - North - Magazines & Newspapers - Circul...   Main Library   
2                            Cheviot Adult Magazines        Cheviot   
3                      North Central Adult Magazines  North Central   
4  Main - North - Magazines & Newspapers - Circul...   Main Library   

  item_callnumber  ... agency_code_num item_status_code item_status_name  \
0      periodical  ...              23                w        WITHDRAWN   
1      periodical  ...               1                w        WITHDRAWN   
2      periodical  ...               6                w        WITHDRAWN   
3      periodical  ...              28                w        WITHDRAWN   
4      periodical  ...               1                -    CHECK SHELVES   

   last_circ_act_date checkout_date due_date checkout_total renewal_total  \
0                None          None     None              0             0   
1                None          None     None              0             0   
2                None          None     None              0             0   
3                None          None     None              0             0   
4                None          None     None              0             0   

  item_price  publish_year  
0        0.0        1963.0  
1        5.0        1963.0  
2        0.0        1963.0  
3        0.0        1894.0  
4        5.0        1894.0  

[5 rows x 23 columns]

In [70]:
#@title
url = 'https://collection-analysis.cincy.pl/current_collection.json?_shape=array&sql=select%0D%0A++item_format%2C%0D%0A++branch_name%2C%0D%0A++count%28%2A%29+as+count_items%0D%0Afrom%0D%0A++item_view%0D%0Awhere%0D%0A++%22checkout_total%22+%3D+%3Ap0%0D%0A++and+%22creation_date%22+%3E%3D+%3Ap1%0D%0A++and+%22creation_date%22+%3C%3D+%3Ap2%0D%0A++and+item_format+not+in+%28%3Ap3%2C+%3Ap4%2C+%3Ap5%2C+%3Ap6%2C+%3Ap7%2C+%3Ap8%29%0D%0Agroup+by%0D%0A++1%2C%0D%0A++2&p1=2021-03-01&p2=2021-09-01&p3=Downloadable+Audiobook&p4=Downloadable+Book&p5=Downloadable+Video&p6=Downloadable+Music&p7=Web+Document&p8=Rare+Book&p0=0'
results = s.get(url=url)

df = pd.json_normalize(results.json())

In [77]:
#@title
df.head()

,item_format,branch_name,count_items
0,Bluray,Main Library,2
1,Book,Anderson,40
2,Book,Avondale,84
3,Book,Blue Ash,60
4,Book,Bond Hill,175


In [83]:
#@title
chart = alt.Chart(df).mark_bar().encode(
    # x='end_date:T',
    x=alt.X(
        'item_format',
        axis=alt.Axis(title='item format', labelAngle=-70),
        # sort='-y'
    ),
    y=alt.Y(
        'count_items:Q',
        title = 'count total items w/ no chk outs (6-12 months old)',
        # scale = alt.Scale(domain=[0,100]),
    ),
    color=alt.Color(
        'item_format',
        # legend=None
    ),
    tooltip=['branch_name', 'item_format', 'count_items']
).properties(
    title="",
    # .format(df.iloc[0]['branch_name']),
    width=400,
).facet(
    facet='branch_name',
    columns=4
)

chart.display()
chart.save('output.html')